#  AE reporting trajectories (remove temporal trend)


## Load data

In [6]:
import itertools
from tqdm import tqdm
from collections import Counter
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# %matplotlib notebook
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')


In [7]:
# load the dictionaries for drugs, SE, indications
drug_dic = pickle.load(open('../Data/high_order/dic/drug_dic_FAERS.pk', 'rb'))  # Save it as dataframe, pandas format
SE_dic = pickle.load(open('../Data/high_order/dic/side_dic_FAERS.pk', 'rb'))  # Save it as dataframe, pandas format
indication_dic = pickle.load(open('../Data/high_order/dic/indication_dic_FAERS.pk', 'rb'))  # Save it as dataframe, pandas format

# In this MeDRA_dic, key is string of PT_name, value is a list:
# [PT, PT_name, HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr]
medra_se_disease_dic = pickle.load(open('../Data/high_order/dic/MeDRA_dic.pk', 'rb'))
len(drug_dic), len(SE_dic), len(indication_dic), len(medra_se_disease_dic)

(3624, 16212, 11851, 24313)

In [8]:
from statsmodels.tsa.ar_model import AutoReg
import math

def standardized_residual(X,Y, y_hat):
    X = np.array(X, dtype=float)
    Y = np.array(Y, dtype=float)
    mean_X = np.mean(X)
    mean_Y = np.mean(Y)
    n = len(X)
    diff_mean_sqr = np.dot((X - mean_X), (X - mean_X))

    residuals = Y - y_hat
    h_ii = (X - mean_X) ** 2 / diff_mean_sqr + (1 / n)
    Var_e = math.sqrt(np.sum((Y - y_hat) ** 2)/(n-2))
    SE_regression = Var_e*((1-h_ii) ** 0.5)
    studentized_residuals = residuals/SE_regression
    return studentized_residuals

## PAEAI based on standarized residual (all for training)

Also called *internally studentized residuals* (which Minitab calls standardized residuals) https://online.stat.psu.edu/stat501/lesson/11/11.3

In [9]:
condition_list = ['SE_uncondition_2019_sig_over', 'SE_uncondition_2019_sig_under', 'SE_male_2019_sig_over', 'SE_male_2019_sig_under',
                 'SE_female_2019_sig_over', 'SE_female_2019_sig_under', 
                 'SE_young_2019_sig_over', 'SE_young_2019_sig_under', 'SE_adult_2019_sig_over', 'SE_adult_2019_sig_under',
                 'SE_elderly_2019_sig_over', 'SE_elderly_2019_sig_under']
for condition in condition_list:    
    pop = condition.split('_')[1]
    # load historical data, calculate SE_tem_new, all reports
    # the SE_uncondition.pk contains all the necessary data
#     locals()['SE_'+pop] = pickle.load(open('../Data/pandemic/results/SE_'+pop+'_historical.pk', 'rb'))
    locals()['SE_'+pop] = pickle.load(open('../Data/pandemic/SE_'+pop+'.pk', 'rb'))
    print('load', pop, len(locals()['SE_'+pop]))

    SE_tem = locals()['SE_'+pop][['SE', '2013_A','2014_A','2015_A','2016_A','2017_A','2018_A','2019_A','2020_A']]
    
    reports_nmb = []
    for year in [2013, 2014, 2015, 2016, 2017, 2018, 2019,2020]:
        year_reports = locals()['SE_'+pop].loc[0][str(year)+'_A'] + locals()['SE_'+pop].loc[0][str(year)+'_B']
        reports_nmb.append(year_reports)
    SE_tem_por = SE_tem.loc[:,"2013_A":"2020_A"].div(reports_nmb, axis=1)
    SE_tem_new = pd.concat([SE_tem.SE, SE_tem_por], axis=1)

    # load sig SE from step 1, the results of step 1
    locals()[condition] = pickle.load(open('../Data/pandemic/results/'+condition+'_step1.pk', 'rb'))    
    se_sig = list(locals()[condition].SE)
    from sklearn.metrics import r2_score
    

    PIC_list = []
    for l in range(len(se_sig)):
        value = SE_tem_new[SE_tem_new.SE == se_sig[l]].to_numpy()[0]
        value = value[1:]

        points = np.array(value).reshape(8,1)   
        train_point = 8    
        yr = np.array(range(1,train_point+1)).reshape(train_point,1)  # yr is the x-axis, from 1 to 8

        lag = 2
        mod = AutoReg(points[:7], lags=lag)
        model_fit = mod.fit()
        predictions = model_fit.predict(start=1, end=7, dynamic=False)
        standardized_residuals = standardized_residual(X=[3,4,5,6,7,8], Y= list(itertools.chain(*points))[2:], y_hat= predictions)

        PIC = abs(standardized_residuals[-1])/(abs(standardized_residuals[:-1]).mean())
        PIC = np.log10(PIC)
        PIC_list.append(PIC)
        r2 = r2_score(list(itertools.chain(*points))[2:-1], predictions[:-1])  # R2-squared score. we can also report this in paper
#         if condition[:-5]=='SE_uncondition_2019_sig':
#             print('PIC, r-squared, AE:', PIC, r2, l)
    ind = [i >0 for i in PIC_list]
    locals()[condition]['PIC'] = PIC_list  # add the PIC
    if condition in ['SE_uncondition_2019_sig_over', 'SE_uncondition_2019_sig_under']: 
        # download all the PIC to draw temporal trend plot 
        pickle.dump(locals()[condition],  open('../Data/pandemic/results/'+condition+'_allPIC.pk', 'wb'))

    
    locals()[condition] = locals()[condition][ind]
    pickle.dump(locals()[condition],  open('../Data/pandemic/results/'+condition+'.pk', 'wb'))
    print(condition, 'saved')

15454
SE_uncondition_2019_sig_over saved
15454
SE_uncondition_2019_sig_under saved
12439
SE_male_2019_sig_over saved
12439
SE_male_2019_sig_under saved
13433
SE_female_2019_sig_over saved
13433
SE_female_2019_sig_under saved
7105
SE_young_2019_sig_over saved
7105
SE_young_2019_sig_under saved
12594
SE_adult_2019_sig_over saved
12594
SE_adult_2019_sig_under saved
10283
SE_elderly_2019_sig_over saved
10283
SE_elderly_2019_sig_under saved


### Sanity check PIC (PAEAI) 

In [10]:
SE_uncondition_2019_sig_over_allPIC = pickle.load(open('../Data/pandemic/results/SE_uncondition_2019_sig_over_allPIC.pk', 'rb'))
SE_uncondition_2019_sig_over_allPIC.head()

,SE,name,2019_A,2019_B,2020_A,2020_B,2019_ROR,2019_Delta,p_value,sig,p_corrected,CI_upper,CI_lower,PIC
153,10001480,ageusia,211,220709,329,210823,1.632358,0.559242,2.373984e-08,True,2.364725e-04,1.940599,1.373078,0.516134
169,10001551,alanine aminotransferase increased,436,220484,1222,209930,2.943658,1.802752,1.523859e-94,True,1.517916e-90,3.284301,2.638346,1.539502
187,10001639,alcoholism,177,220743,267,210885,1.578990,0.508475,2.348872e-06,True,2.339711e-02,1.909531,1.305665,0.297520
307,10002511,anhedonia,24,220896,79,211073,3.444856,2.291667,8.937515e-09,True,8.902659e-05,5.439852,2.181499,1.102174
320,10002556,ankylosing spondylitis,34,220886,82,211070,2.523926,1.411765,2.455387e-06,True,2.445811e-02,3.764655,1.692108,0.292150


## Draw and check figures

Uncomment if you want to check the difference between observation and expectation. 

In [11]:
# se_sig = SE_elderly_2019_sig_over.SE
# PIC_list = []
# for l in range(len(se_sig)):
#     value = SE_tem_new[SE_tem_new.SE == list(se_sig)[l]].to_numpy()[0]
#     value = value[1:]

#     points = np.array(value).reshape(8,1)   
#     train_point = 8    
#     yr = np.array(range(1,train_point+1)).reshape(train_point,1)  # yr is the x-axis, from 1 to 8

#     lag = 2
#     mod = AutoReg(points[:7], lags=lag)
#     model_fit = mod.fit()
#     predictions = model_fit.predict(start=1, end=7, dynamic=False)

#     plt.plot(range(lag, 8),predictions)
#     plt.plot( points, color='red')
#     plt.title('linear regression: red (true), blue (regression)')
#     plt.xlabel('year')
#     plt.ylabel('percentage')
#     plt.show()

# Group PT to SOC 

Dased on MedDRA hierarchy.

PT (preferred terms)--> HLT (high-level term) --> HLGT (high-level group term) --> SOC (system organ class)

In [12]:
MedDRA_dic_all = pickle.load(open('../Data/high_order/dic/MedDRA_dic_all.pk', 'rb'))

In [13]:
# MedDRA_dic_all[MedDRA_dic_all.PT_name == row.name].SOC
condition_list = ['SE_uncondition_2019_sig_over', 'SE_uncondition_2019_sig_under', 'SE_male_2019_sig_over', 'SE_male_2019_sig_under',
                 'SE_female_2019_sig_over', 'SE_female_2019_sig_under', 
                 'SE_young_2019_sig_over', 'SE_young_2019_sig_under', 'SE_adult_2019_sig_over', 'SE_adult_2019_sig_under',
                 'SE_elderly_2019_sig_over', 'SE_elderly_2019_sig_under']

for condition in condition_list:      
    print('condition', condition)
    locals()[condition] = pickle.load(open('../Data/pandemic/results/'+condition+'.pk', 'rb'))
    
    

    
    locals()[condition]['SOC'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC, axis = 1)
    locals()[condition]['SOC_name'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC_name, axis = 1)
    locals()[condition]['SOC_abbr'] = locals()[condition].apply(lambda row: MedDRA_dic_all[MedDRA_dic_all.PT_name == row['name']].SOC_abbr, axis = 1)

    #     Save the final dataframe that with SOC information
    pickle.dump(locals()[condition],  open('../Data/pandemic/results/'+condition+'_step2.pk', 'wb'))
    print(condition, 'saved')

condition SE_uncondition_2019_sig_over
SE_uncondition_2019_sig_over saved
condition SE_uncondition_2019_sig_under
SE_uncondition_2019_sig_under saved
condition SE_male_2019_sig_over
SE_male_2019_sig_over saved
condition SE_male_2019_sig_under
SE_male_2019_sig_under saved
condition SE_female_2019_sig_over
SE_female_2019_sig_over saved
condition SE_female_2019_sig_under
SE_female_2019_sig_under saved
condition SE_young_2019_sig_over
SE_young_2019_sig_over saved
condition SE_young_2019_sig_under
SE_young_2019_sig_under saved
condition SE_adult_2019_sig_over
SE_adult_2019_sig_over saved
condition SE_adult_2019_sig_under
SE_adult_2019_sig_under saved
condition SE_elderly_2019_sig_over
SE_elderly_2019_sig_over saved
condition SE_elderly_2019_sig_under
SE_elderly_2019_sig_under saved
